In [ ]:
# Heart Disease UCI Dataset - PCA Analysis
# Dimensionality Reduction using Principal Component Analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('default')
sns.set_palette("husl")

print("=== Heart Disease Prediction - PCA Analysis ===")
print("Performing Principal Component Analysis for dimensionality reduction...")

# Load preprocessed data
try:
    X_scaled = pd.read_csv('../data/X_scaled.csv')
    y = pd.read_csv('../data/y.csv')['target']
    print("✅ Preprocessed data loaded successfully")
except FileNotFoundError:
    print("❌ Preprocessed data not found. Please run 01_data_preprocessing.ipynb first.")
    # Create sample data for demonstration
    print("Creating sample data for demonstration...")
    np.random.seed(42)
    n_samples, n_features = 303, 13
    X_scaled = pd.DataFrame(
        np.random.randn(n_samples, n_features),
        columns=[f'feature_{i}' for i in range(n_features)]
    )
    y = pd.Series(np.random.choice([0, 1], n_samples), name='target')
    print("✅ Sample data created")

print(f"\nDataset shape: {X_scaled.shape}")
print(f"Features: {list(X_scaled.columns)}")

# 1. INITIAL PCA ANALYSIS
print("\n" + "="*60)
print("1. INITIAL PCA ANALYSIS")
print("="*60)

# Apply PCA with all components first
pca_full = PCA()
X_pca_full = pca_full.fit_transform(X_scaled)

print(f"Original number of features: {X_scaled.shape[1]}")
print(f"Explained variance ratio for each component:")
for i, ratio in enumerate(pca_full.explained_variance_ratio_):
    print(f"  PC{i+1}: {ratio:.4f} ({ratio*100:.2f}%)")

# Cumulative explained variance
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
print(f"\nCumulative explained variance:")
for i, cum_var in enumerate(cumulative_variance):
    print(f"  First {i+1} PCs: {cum_var:.4f} ({cum_var*100:.2f}%)")

# 2. VISUALIZATION OF EXPLAINED VARIANCE
print("\n" + "="*60)
print("2. EXPLAINED VARIANCE VISUALIZATION")
print("="*60)

# Create comprehensive PCA visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Principal Component Analysis - Heart Disease Dataset', fontsize=16, fontweight='bold')

# Individual explained variance
axes[0,0].bar(range(1, len(pca_full.explained_variance_ratio_) + 1),
              pca_full.explained_variance_ratio_)
axes[0,0].set_xlabel('Principal Component')
axes[0,0].set_ylabel('Explained Variance Ratio')
axes[0,0].set_title('Explained Variance by Component')
axes[0,0].grid(True, alpha=0.3)

# Cumulative explained variance
axes[0,1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance,
               marker='o', linewidth=2, markersize=6)
axes[0,1].axhline(y=0.95, color='red', linestyle='--', alpha=0.7, label='95% Variance')
axes[0,1].axhline(y=0.90, color='orange', linestyle='--', alpha=0.7, label='90% Variance')
axes[0,1].axhline(y=0.85, color='green', linestyle='--', alpha=0.7, label='85% Variance')
axes[0,1].set_xlabel('Number of Components')
axes[0,1].set_ylabel('Cumulative Explained Variance')
axes[0,1].set_title('Cumulative Explained Variance')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Scree plot
axes[1,0].plot(range(1, len(pca_full.explained_variance_) + 1),
               pca_full.explained_variance_, marker='o', linewidth=2, markersize=6)
axes[1,0].set_xlabel('Principal Component')
axes[1,0].set_ylabel('Eigenvalue')
axes[1,0].set_title('Scree Plot')
axes[1,0].grid(True, alpha=0.3)

# First two principal components scatter plot
scatter = axes[1,1].scatter(X_pca_full[:, 0], X_pca_full[:, 1],
                           c=y, cmap='coolwarm', alpha=0.7)
axes[1,1].set_xlabel(f'PC1 ({pca_full.explained_variance_ratio_[0]:.2%} variance)')
axes[1,1].set_ylabel(f'PC2 ({pca_full.explained_variance_ratio_[1]:.2%} variance)')
axes[1,1].set_title('First Two Principal Components')
plt.colorbar(scatter, ax=axes[1,1], label='Heart Disease')

plt.tight_layout()
plt.show()

# 3. DETERMINE OPTIMAL NUMBER OF COMPONENTS
print("\n" + "="*60)
print("3. DETERMINING OPTIMAL NUMBER OF COMPONENTS")
print("="*60)

# Different criteria for selecting components
criteria = {
    '95% variance': 0.95,
    '90% variance': 0.90,
    '85% variance': 0.85
}

optimal_components = {}
for criterion, threshold in criteria.items():
    n_components = np.argmax(cumulative_variance >= threshold) + 1
    optimal_components[criterion] = n_components
    print(f"📊 {criterion}: {n_components} components needed")

# Kaiser criterion (eigenvalue > 1)
eigenvalues = pca_full.explained_variance_
kaiser_components = np.sum(eigenvalues > 1)
optimal_components['Kaiser criterion'] = kaiser_components
print(f"📊 Kaiser criterion (eigenvalue > 1): {kaiser_components} components")

# Elbow method visualization
def find_elbow_point(variances):
    """Find elbow point using the rate of change method"""
    n = len(variances)
    if n < 3:
        return 1

    # Calculate second derivative (rate of change of rate of change)
    second_derivatives = []
    for i in range(1, n-1):
        second_derivative = variances[i-1] - 2*variances[i] + variances[i+1]
        second_derivatives.append(abs(second_derivative))

    # Find the point with maximum second derivative
    if second_derivatives:
        elbow_index = np.argmax(second_derivatives) + 1  # +1 to adjust for indexing
        return elbow_index + 1  # +1 because components start from 1
    return 2

elbow_components = find_elbow_point(pca_full.explained_variance_ratio_)
optimal_components['Elbow method'] = elbow_components
print(f"📊 Elbow method: {elbow_components} components")

print(f"\n🎯 Summary of optimal components:")
for method, n_comp in optimal_components.items():
    variance_explained = cumulative_variance[n_comp-1]
    print(f"  {method}: {n_comp} components ({variance_explained:.2%} variance)")

# 4. APPLY PCA WITH SELECTED NUMBER OF COMPONENTS
print("\n" + "="*60)
print("4. APPLYING PCA WITH OPTIMAL COMPONENTS")
print("="*60)

# Use the 90% variance criterion as default
n_components_selected = optimal_components['90% variance']
print(f"🔧 Using {n_components_selected} principal components (90% variance criterion)")

# Apply PCA with selected components
pca_selected = PCA(n_components=n_components_selected)
X_pca_selected = pca_selected.fit_transform(X_scaled)

print(f"✅ PCA applied successfully!")
print(f"📊 Original dimensions: {X_scaled.shape}")
print(f"📊 Reduced dimensions: {X_pca_selected.shape}")
print(f"📊 Dimensionality reduction: {X_scaled.shape[1]} → {X_pca_selected.shape[1]}")
print(f"📊 Variance retained: {np.sum(pca_selected.explained_variance_ratio_):.2%}")

# Create DataFrame with PCA components
pca_columns = [f'PC{i+1}' for i in range(n_components_selected)]
X_pca_df = pd.DataFrame(X_pca_selected, columns=pca_columns)

print(f"\n📈 PCA components statistics:")
print(X_pca_df.describe())

# 5. COMPONENT INTERPRETATION
print("\n" + "="*60)
print("5. PRINCIPAL COMPONENT INTERPRETATION")
print("="*60)

# Feature loadings (components)
feature_names = X_scaled.columns
loadings = pca_selected.components_

print("📊 Feature loadings for principal components:")
loadings_df = pd.DataFrame(
    loadings.T,
    columns=[f'PC{i+1}' for i in range(n_components_selected)],
    index=feature_names
)
print(loadings_df.round(3))

# Visualize feature loadings
fig, axes = plt.subplots(1, min(3, n_components_selected), figsize=(15, 5))
if n_components_selected == 1:
    axes = [axes]
elif n_components_selected == 2:
    axes = axes.flatten()

for i in range(min(3, n_components_selected)):
    loadings_sorted = loadings_df[f'PC{i+1}'].abs().sort_values(ascending=True)
    colors = ['red' if x < 0 else 'blue' for x in loadings_df.loc[loadings_sorted.index, f'PC{i+1}']]

    axes[i].barh(range(len(loadings_sorted)),
                 loadings_df.loc[loadings_sorted.index, f'PC{i+1}'],
                 color=colors, alpha=0.7)
    axes[i].set_yticks(range(len(loadings_sorted)))
    axes[i].set_yticklabels(loadings_sorted.index, fontsize=10)
    axes[i].set_xlabel('Loading')
    axes[i].set_title(f'PC{i+1} Feature Loadings\n({pca_selected.explained_variance_ratio_[i]:.1%} variance)')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 6. PCA VISUALIZATION IN 2D AND 3D
print("\n" + "="*60)
print("6. PCA VISUALIZATION")
print("="*60)

if n_components_selected >= 2:
    # 2D visualization
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    scatter = plt.scatter(X_pca_selected[:, 0], X_pca_selected[:, 1],
                         c=y, cmap='coolwarm', alpha=0.7, s=50)
    plt.xlabel(f'PC1 ({pca_selected.explained_variance_ratio_[0]:.1%} variance)')
    plt.ylabel(f'PC2 ({pca_selected.explained_variance_ratio_[1]:.1%} variance)')
    plt.title('Heart Disease Data in PC1-PC2 Space')
    plt.colorbar(scatter, label='Heart Disease (0=No, 1=Yes)')
    plt.grid(True, alpha=0.3)

    # Density plot
    plt.subplot(1, 2, 2)
    for target_class in [0, 1]:
        mask = y == target_class
        plt.scatter(X_pca_selected[mask, 0], X_pca_selected[mask, 1],
                   alpha=0.6, s=50, label=f'Class {target_class}')
    plt.xlabel(f'PC1 ({pca_selected.explained_variance_ratio_[0]:.1%} variance)')
    plt.ylabel(f'PC2 ({pca_selected.explained_variance_ratio_[1]:.1%} variance)')
    plt.title('Heart Disease Classes in PCA Space')
    plt.legend(['No Disease', 'Disease'])
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

if n_components_selected >= 3:
    # 3D visualization
    from mpl_toolkits.mplot3d import Axes3D

    fig = plt.figure(figsize=(12, 5))

    ax1 = fig.add_subplot(121, projection='3d')
    scatter = ax1.scatter(X_pca_selected[:, 0], X_pca_selected[:, 1], X_pca_selected[:, 2],
                         c=y, cmap='coolwarm', alpha=0.6, s=50)
    ax1.set_xlabel(f'PC1 ({pca_selected.explained_variance_ratio_[0]:.1%})')
    ax1.set_ylabel(f'PC2 ({pca_selected.explained_variance_ratio_[1]:.1%})')
    ax1.set_zlabel(f'PC3 ({pca_selected.explained_variance_ratio_[2]:.1%})')
    ax1.set_title('3D PCA Visualization')

    ax2 = fig.add_subplot(122, projection='3d')
    for target_class in [0, 1]:
        mask = y == target_class
        ax2.scatter(X_pca_selected[mask, 0], X_pca_selected[mask, 1], X_pca_selected[mask, 2],
                   alpha=0.6, s=50, label=f'Class {target_class}')
    ax2.set_xlabel(f'PC1 ({pca_selected.explained_variance_ratio_[0]:.1%})')
    ax2.set_ylabel(f'PC2 ({pca_selected.explained_variance_ratio_[1]:.1%})')
    ax2.set_zlabel(f'PC3 ({pca_selected.explained_variance_ratio_[2]:.1%})')
    ax2.set_title('3D PCA by Class')
    ax2.legend(['No Disease', 'Disease'])

    plt.tight_layout()
    plt.show()

# 7. COMPARISON: ORIGINAL VS PCA DATA
print("\n" + "="*60)
print("7. ORIGINAL VS PCA DATA COMPARISON")
print("="*60)

print("📊 Data dimensionality comparison:")
print(f"  Original data: {X_scaled.shape[1]} features")
print(f"  PCA data: {X_pca_selected.shape[1]} components")
print(f"  Reduction ratio: {(1 - X_pca_selected.shape[1]/X_scaled.shape[1]):.1%}")

print(f"\n📊 Variance retention:")
total_variance_retained = np.sum(pca_selected.explained_variance_ratio_)
print(f"  Variance retained: {total_variance_retained:.2%}")
print(f"  Variance lost: {(1 - total_variance_retained):.2%}")

# Statistical comparison
print(f"\n📊 Statistical properties:")
print("Original data:")
print(f"  Mean: {X_scaled.mean().mean():.4f}")
print(f"  Std: {X_scaled.std().mean():.4f}")
print("PCA data:")
print(f"  Mean: {X_pca_df.mean().mean():.4f}")
print(f"  Std: {X_pca_df.std().mean():.4f}")

# 8. SAVE PCA RESULTS
print("\n" + "="*60)
print("8. SAVING PCA RESULTS")
print("="*60)

try:
    import os
    os.makedirs('../data', exist_ok=True)

    # Save PCA transformed data
    X_pca_df.to_csv('data/X_pca.csv', index=False)

    # Save PCA model parameters
    import joblib
    joblib.dump(pca_selected, '../models/pca_model.pkl')

    # Save PCA analysis results
    pca_results = {
        'n_components': n_components_selected,
        'explained_variance_ratio': pca_selected.explained_variance_ratio_.tolist(),
        'explained_variance': pca_selected.explained_variance_.tolist(),
        'components': pca_selected.components_.tolist(),
        'feature_names': feature_names.tolist(),
        'total_variance_retained': float(total_variance_retained)
    }

    import json
    with open('../results/pca_analysis.json', 'w') as f:
        json.dump(pca_results, f, indent=2)

    # Save loadings matrix
    loadings_df.to_csv('results/pca_loadings.csv')

    print("✅ PCA results saved successfully!")
    print("Files saved:")
    print("  - X_pca.csv (PCA transformed data)")
    print("  - pca_model.pkl (PCA model)")
    print("  - pca_analysis.json (PCA results)")
    print("  - pca_loadings.csv (Feature loadings)")

except Exception as e:
    print(f"⚠️ Error saving files: {e}")
    print("Creating necessary directories...")
    os.makedirs('../models', exist_ok=True)
    os.makedirs('../results', exist_ok=True)

# 9. PCA ANALYSIS SUMMARY
print("\n" + "="*60)
print("9. PCA ANALYSIS SUMMARY")
print("="*60)

print("✅ PCA analysis completed successfully!")
print(f"📊 Dimensionality reduction: {X_scaled.shape[1]} → {n_components_selected} features")
print(f"📈 Variance retained: {total_variance_retained:.2%}")
print(f"🎯 Selected components explain {total_variance_retained:.1%} of total variance")

print(f"\n📋 Top contributing features per component:")
for i in range(min(3, n_components_selected)):
    pc_name = f'PC{i+1}'
    top_features = loadings_df[pc_name].abs().nlargest(3)
    print(f"  {pc_name} ({pca_selected.explained_variance_ratio_[i]:.1%} variance):")
    for feature, loading in top_features.items():
        direction = "+" if loadings_df.loc[feature, pc_name] > 0 else "-"
        print(f"    {direction} {feature} ({abs(loadings_df.loc[feature, pc_name]):.3f})")

print(f"\n🔍 Component interpretation guidelines:")
print("  - PC1: Usually captures the most significant pattern in data")
print("  - PC2: Captures the second most important orthogonal pattern")
print("  - Higher PCs: Capture remaining variance, often noise")

print(f"\n🎯 Next steps:")
print("  1. ✅ Data preprocessing complete")
print("  2. ✅ PCA analysis complete")
print("  3. ⏳ Feature selection (03_feature_selection.ipynb)")
print("  4. ⏳ Train supervised learning models (04_supervised_learning.ipynb)")
print("  5. ⏳ Apply unsupervised learning (05_unsupervised_learning.ipynb)")
print("  6. ⏳ Hyperparameter tuning (06_hyperparameter_tuning.ipynb)")

print(f"\n💡 Recommendations:")
print(f"  - Use {n_components_selected} components for modeling to retain {total_variance_retained:.1%} variance")
print("  - Consider both original and PCA-transformed features for model comparison")
print("  - Monitor model performance difference between original and reduced data")

print(f"\n🎉 Ready to proceed to feature selection!")

# Display final PCA summary table
print(f"\n📊 Final PCA Components Summary:")
summary_data = []
for i in range(n_components_selected):
    summary_data.append([
        f'PC{i+1}',
        f'{pca_selected.explained_variance_ratio_[i]:.3f}',
        f'{pca_selected.explained_variance_ratio_[i]*100:.1f}%',
        f'{cumulative_variance[i]*100:.1f}%'
    ])

summary_df = pd.DataFrame(summary_data,
                         columns=['Component', 'Variance Ratio', 'Individual %', 'Cumulative %'])
print(summary_df.to_string(index=False))